In [184]:
import pandas as pd
import numpy as np

# Define a opção para mostrar TODAS as colunas, sem limite
pd.set_option('display.max_columns', None)

# Define a opção para mostrar TODAS as linhas, sem limite
pd.set_option('display.max_rows', None)


# Define a opção para mostrar o número máximo de elementos do array como infinito (None)
np.set_printoptions(threshold=np.inf)

In [185]:

df = pd.read_csv("../datasets/dataframe_yuri_xiaomi_x5.csv")
df.shape

(2090, 20)

In [186]:
X = df.drop(columns = ['Device', 'Brand', 'Model', 'Created', 'RP'])
Y = df["RP"]

print(X.shape)
print(Y.shape)

(2090, 15)
(2090,)


In [187]:
X['Direction'] = X['Direction'].replace('norte', 1)
X['Direction'] = X['Direction'].replace('leste', 2)
X['Direction'] = X['Direction'].replace('sul', 3)
X['Direction'] = X['Direction'].replace('oeste', 4)



/var/folders/jl/02kt03gj5n99h1czkswp582c0000gn/T/ipykernel_29498/849246506.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X['Direction'] = X['Direction'].replace('oeste', 4)


Bibliotecas para o treinamento

In [205]:
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import random

In [189]:
X = X.to_numpy()
Y = Y.to_numpy()


In [190]:
lower_limit = 1
upper_limit = 100

seed_labels = ["seed1", "seed2", "seed3", "seed4", "seed5"]

sets_division = {
    label: {"seed":"", "X_train":"", "X_test":"", "Y_train":"", "Y_test":""}
    for label in seed_labels
}


random_state = [random.randint(lower_limit, upper_limit) for _ in range(len(sets_division))]
print(f"Sementes aletórios para divisão do treino: {random_state}")


index_seed = 0
for seed, set_division in sets_division.items():
   print(f"Iniciando a divisão com a semente {seed}: {random_state[index_seed]}")

   set_division["seed"] = random_state[index_seed]
   set_division["X_train"], set_division["X_test"], set_division["Y_train"], set_division["Y_test"] = train_test_split(X, Y, test_size=0.3, random_state=random_state[index_seed], stratify=Y)

   index_seed += 1



Sementes aletórios para divisão do treino: [85, 60, 58, 55, 44]
Iniciando a divisão com a semente seed1: 85
Iniciando a divisão com a semente seed2: 60
Iniciando a divisão com a semente seed3: 58
Iniciando a divisão com a semente seed4: 55
Iniciando a divisão com a semente seed5: 44


In [191]:
cv = StratifiedKFold(n_splits = 5, shuffle = True, random_state=42)
param_grid = {'n_neighbors':[1,3,5,7,9,11]}
knn_clf = KNeighborsClassifier()

In [193]:
def get_best_k_model(model, x_fit, y_fit, parametros, estrategia_treinamento, reference_metric, processos_de_cpu):
    
    grid_search_class = GridSearchCV(estimator=model, param_grid=parametros, cv=estrategia_treinamento, scoring=reference_metric, n_jobs = processos_de_cpu)
    grid_search_class.fit(x_fit, y_fit)
    best_k = grid_search_class.best_params_['n_neighbors']
    best_knn_clf = grid_search_class.best_estimator_
    
    return  best_k, best_knn_clf


'''
 TESTE PARA VER SE ESTAVA FUNCIONANDO
 
X_train = sets_division["seed1"]["X_train"]
X_test = sets_division["seed1"]["X_test"]
Y_train = sets_division["seed1"]["Y_train"]
Y_test = sets_division["seed1"]["Y_test"]

print(f"(SHAPES)\nX_train: {X_train.shape}; \nX_test: {X_test.shape}; \nY_train: {Y_train.shape}; \nY_test: {Y_test.shape}")

best_k, best_knn = get_best_k_model(model=knn_clf, x_fit=X_train, y_fit=Y_train, parametros=param_grid,
                                    estrategia_treinamento=cv, reference_metric="accuracy", processos_de_cpu=1)

print(f"Melhor k encontrado para o KNN: {best_k}")
print(best_knn)
'''

'\n TESTE PARA VER SE ESTAVA FUNCIONANDO\n \nX_train = sets_division["seed1"]["X_train"]\nX_test = sets_division["seed1"]["X_test"]\nY_train = sets_division["seed1"]["Y_train"]\nY_test = sets_division["seed1"]["Y_test"]\n\nprint(f"(SHAPES)\nX_train: {X_train.shape}; \nX_test: {X_test.shape}; \nY_train: {Y_train.shape}; \nY_test: {Y_test.shape}")\n\nbest_k, best_knn = get_best_k_model(model=knn_clf, x_fit=X_train, y_fit=Y_train, parametros=param_grid,\n                                    estrategia_treinamento=cv, reference_metric="accuracy", processos_de_cpu=1)\n\nprint(f"Melhor k encontrado para o KNN: {best_k}")\nprint(best_knn)\n'

# Predicao

conjunto dos valores x y para cada posicao de classificacao P_X_Y

In [195]:
df_rp = df["RP"].to_numpy()
categories = np.unique(df_rp)

sets_positions = {'X':{}, "Y":{}}

for categorie in categories:
    
    first_record_category = df.loc[df['RP'] == categorie].iloc[0]
    
    y = float(first_record_category["Y"])
    x = float(first_record_category["X"])


    split_categorie = categorie.split("_")
    #print(split_categorie)

    if(split_categorie[1] not in sets_positions["Y"]):
        #print(f"Adicionando o valor {y} associado a P_{split_categorie[1]}_x")
        sets_positions["Y"][f"{split_categorie[1]}"] = y

    if(split_categorie[2] not in sets_positions["X"]):
        #print(f"Adicionando o valor {x} associado a P_Y_{split_categorie[2]}")
        sets_positions["X"][f"{split_categorie[2]}"] = x
    


print(sets_positions["X"])
print(sets_positions["Y"])


{'1': 1.57, '2': 3.92, '3': 6.27}
{'1': 0.77, '2': 2.03, '3': 3.76, '4': 5.42, '5': 7.09, '6': 8.75, '7': 10.42, '8': 12.08, '9': 13.74}


In [196]:
def converte_categoria_posicao(values_pred,values_real):
    posicoes_pred = []
    posicoes_real = []
    shape_pred = values_pred.shape[0]
    shape_real = values_real.shape[0]

    if(shape_pred != shape_real):
        print(f"Erro a dimensão dos valores previstos é diferente da dimensão dos valores reais: (pred {values_pred}) (real {values_real})")
        return
    
    for index in range(len(values_real)):

        value_pred_split = values_pred[index].split("_")
        value_real_split = values_real[index].split("_")

        x_pred = sets_positions['X'][value_pred_split[2]]
        y_pred = sets_positions['Y'][value_pred_split[1]]
        x_real = sets_positions['X'][value_real_split[2]]
        y_real = sets_positions['Y'][value_real_split[1]]

        posicoes_pred.append((x_pred, y_pred))
        posicoes_real.append((x_real, y_real))

    return posicoes_pred, posicoes_real


In [209]:
set_positions_seed = {}
set_test_pred_categories = {}

for seed, division in sets_division.items():

    print(f"\n===================== TREINAMENTO COM BASE NA DIVISÃO DA SEMENTE {seed}: {division['seed']} =======================")

    #Divisao dos dados de acordo como foi dividido por cada semente
    X_train = division["X_train"]
    X_test = division["X_test"]
    Y_train = division["Y_train"]
    Y_test = division["Y_test"]

    print(f"(SHAPES)\nX_train: {X_train.shape}; \nX_test: {X_test.shape}; \nY_train: {Y_train.shape}; \nY_test: {Y_test.shape};")

    #Busca em grade para o melhor K, e modelo do KNN com base na acurácia, e nos dados de treino e teste
    best_k, best_knn = get_best_k_model(model=knn_clf, x_fit=X_train, y_fit=Y_train, parametros=param_grid,
                                        estrategia_treinamento=cv, reference_metric="accuracy", processos_de_cpu=1)
    print(f"\nMelhor k encontrado para o KNN: {best_k}")


    #Predicao dos valores utilizando o modelo selecionado pela busca em grade pelo melhor modelo
    Y_pred = best_knn.predict(X_test)
    set_test_pred_categories[seed] =  {'seed':division['seed'],'values_pred':Y_pred, 'values_test':Y_test}

    # Conversão dos valores da categorias para x e y para poder fazer os calculos
    positions_pred, positions_test = converte_categoria_posicao(Y_pred, Y_test)
    preditos = np.array(positions_pred)
    reais = np.array(positions_test)
    
    #adicionando em uma estrutura para poder fazer os plots para melhor analise do desempenho do modelo
    set_positions_seed[seed] = {'seed':division['seed'],'positions_pred':preditos, 'positions_test':reais}
    
    #calculo do erro medio da distancia euclidiana entre os pontos 
    # Di = raiz((Xi_pred - Xi_real)ˆ2 + (Yi_pred - Yireal)ˆ2)
    # MAE = (|D1| + ... + |DN|)/N

    distancias = np.sqrt((preditos[:, 0] - reais[:, 0])**2 + (preditos[:, 1] - reais[:, 1])**2)
    mae = np.mean(distancias)


    print(f"\nErro médio absoluto da distância euclidiana entre os valores preditos e os valores reais: {mae:.2f}")


    print("-"*60)
    print(f"RESULTADOS DA CLASSIFICAÇÃO (PR) - Melhor K: {best_k}")
    print("-"*60)

    #Métricas para a análise com base nos dados preditos pelo modelo e valores reais do test
    print(classification_report(Y_test, Y_pred))


===================== TREINAMENTO COM BASE NA DIVISÃO DA SEMENTE seed1: 85 =======================
(SHAPES)
X_train: (1463, 15); 
X_test: (627, 15); 
Y_train: (1463,); 
Y_test: (627,);

Melhor k encontrado para o KNN: 1

Erro médio absoluto da distância euclidiana entre os valores preditos e os valores reais: 0.36
------------------------------------------------------------
RESULTADOS DA CLASSIFICAÇÃO (PR) - Melhor K: 1
------------------------------------------------------------
              precision    recall  f1-score   support

       P_1_1       0.92      1.00      0.96        35
       P_1_2       0.97      1.00      0.99        35
       P_1_3       1.00      1.00      1.00        35
       P_2_1       0.97      0.94      0.96        35
       P_2_2       1.00      0.97      0.99        35
       P_2_3       0.97      0.97      0.97        35
       P_3_1       0.93      0.65      0.76        20
       P_3_2       0.81      0.85      0.83        20
       P_3_3       0.95    

Pyplots sobre os dados preditos e reais

In [216]:


for seed, valores in set_test_pred_categories.items():

    print(f"\n=============== Analise dos resultados da semente {seed}: {valores['seed']} ===============")
    y_true = valores['values_test']
    y_pred = valores['values_pred']

    labels = np.unique(y_true)
    cm = confusion_matrix(y_true, y_pred, labels=labels)


    # Cria dicionário para armazenar as métricas individuais
    metrics_individuais = {}

    for i, label in enumerate(labels):
        TP = cm[i, i]
        FP = cm[:, i].sum() - TP
        FN = cm[i, :].sum() - TP
        TN = cm.sum() - (TP + FP + FN)
        
        Precision = TP / (TP + FP) if (TP + FP) > 0 else 0
        Recall = TP / (TP + FN) if (TP + FN) > 0 else 0
        F1 = (2 * Precision * Recall) / (Precision + Recall) if (Precision + Recall) > 0 else 0
        Support = cm[i, :].sum()
        
        metrics_individuais[label] = {
            'TP': TP, 'FP': FP, 'FN': FN, 'TN': TN,
            'Precision': round(Precision, 3),
            'Recall': round(Recall, 3),
            'F1-Score': round(F1, 3),
            'Support': Support
        }

    # Converte em DataFrame para visualização
    df_metrics = pd.DataFrame(metrics_individuais).T
    df_metrics = df_metrics.sort_index()
    print(df_metrics)


=============== Analise dos resultados da semente seed1: 85 ===============
         TP    FP   FN     TN  Precision  Recall  F1-Score  Support
P_1_1  35.0   3.0  0.0  589.0      0.921   1.000     0.959     35.0
P_1_2  35.0   1.0  0.0  591.0      0.972   1.000     0.986     35.0
P_1_3  35.0   0.0  0.0  592.0      1.000   1.000     1.000     35.0
P_2_1  33.0   1.0  2.0  591.0      0.971   0.943     0.957     35.0
P_2_2  34.0   0.0  1.0  592.0      1.000   0.971     0.986     35.0
P_2_3  34.0   1.0  1.0  591.0      0.971   0.971     0.971     35.0
P_3_1  13.0   1.0  7.0  606.0      0.929   0.650     0.765     20.0
P_3_2  17.0   4.0  3.0  603.0      0.810   0.850     0.829     20.0
P_3_3  20.0   1.0  0.0  606.0      0.952   1.000     0.976     20.0
P_4_1  14.0   5.0  6.0  602.0      0.737   0.700     0.718     20.0
P_4_2  19.0   4.0  1.0  603.0      0.826   0.950     0.884     20.0
P_4_3  17.0   2.0  3.0  605.0      0.895   0.850     0.872     20.0
P_5_1  11.0  10.0  9.0  597.0      0.52